# Imports

In [3]:
%load_ext autoreload
%autoreload 2
from tqdm.notebook import tqdm
import numpy as np
import pickle
import bigram
from utils import emotion_scores
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

C:\Users\Deeptanshu Barman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Deeptanshu Barman\.cache\huggingface\hub\models--bhadresh-savani--distilbert-base-uncased-emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-developme

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

C:\Users\Deeptanshu Barman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


# Model Setup

In [4]:
corpus_file_path = '../data/corpus.txt'
corpus = open(corpus_file_path, 'r').read()
print(len(corpus))

238101


In [5]:
print(emotion_scores("lmao"))

[{'label': 'sadness', 'score': 0.04048538580536842}, {'label': 'joy', 'score': 0.27616098523139954}, {'label': 'love', 'score': 0.006484142038971186}, {'label': 'anger', 'score': 0.5209841728210449}, {'label': 'fear', 'score': 0.14821961522102356}, {'label': 'surprise', 'score': 0.007665676064789295}]


In [6]:
model=bigram.BigramLM(corpus)
model.set_token()
model.calculate_bigrams()
laplace_matrix = model.get_laplace_matrix()
kn_matrix = model.set_kn_matrix(d = 0.48)

Model Initialized 🟢
Tokens Set 🟢


Populating Bigram Matrix...:   0%|          | 0/46531 [00:00<?, ?it/s]

All Matrices Calculated 🟢


Calculating Kneser-Ney Matrix...:   0%|          | 0/5429 [00:00<?, ?it/s]

# Generating Emotion for all non-zero Bigrams

In [7]:
tokens = model.get_token()
count_matrix = model.get_count_matrix()

In [47]:
emotions = {}
for i, token in tqdm(enumerate(tokens)):
    for j, token2 in enumerate(tokens):
        if count_matrix[i][j] > 0:
            emotions[(token, token2)] = emotion_scores(str(token + " " + token2))

with open('emotions.pkl', 'wb') as f:
    pickle.dump(emotions, f)

0it [00:00, ?it/s]

In [52]:
emotions

{('a', 'a'): [{'label': 'sadness', 'score': 0.04309724643826485},
  {'label': 'joy', 'score': 0.07374849915504456},
  {'label': 'love', 'score': 0.007005644030869007},
  {'label': 'anger', 'score': 0.5979116559028625},
  {'label': 'fear', 'score': 0.2686230540275574},
  {'label': 'surprise', 'score': 0.00961387436836958}],
 ('a', 'acceptable'): [{'label': 'sadness', 'score': 0.004085512366145849},
  {'label': 'joy', 'score': 0.9833838939666748},
  {'label': 'love', 'score': 0.001721488544717431},
  {'label': 'anger', 'score': 0.007491974625736475},
  {'label': 'fear', 'score': 0.0023463102988898754},
  {'label': 'surprise', 'score': 0.0009707380668260157}],
 ('a', 'baachan'): [{'label': 'sadness', 'score': 0.021032186225056648},
  {'label': 'joy', 'score': 0.07375523447990417},
  {'label': 'love', 'score': 0.005947881378233433},
  {'label': 'anger', 'score': 0.8438517451286316},
  {'label': 'fear', 'score': 0.05099637806415558},
  {'label': 'surprise', 'score': 0.004416525363922119}],


# Generating 50 Sentences each of every emotion


In [10]:
emotions=["sadness","joy","love","anger","fear","surprise"]
for i in emotions:
    #used count matrix use some other matrix like laplace or kneser-nay
    model.generate_sentences(model.get_normal_matrix(),emotion=i,word_limit=10,no_of_sentences=50)

Generating sadness Matix: 0it [00:00, ?it/s]

Generating Sentence:   0%|          | 0/50 [00:00<?, ?it/s]

Sentences Generated + Stored 🟢


Generating joy Matix: 0it [00:00, ?it/s]

Generating Sentence:   0%|          | 0/50 [00:00<?, ?it/s]

...

Generating Sentence:   0%|          | 0/50 [00:00<?, ?it/s]

Sentences Generated + Stored 🟢


Generating surprise Matix: 0it [00:00, ?it/s]

Generating Sentence:   0%|          | 0/50 [00:00<?, ?it/s]

Sentences Generated + Stored 🟢


In [66]:
# convert the sentence to a string
sentence = ' '.join(sentence)
print(emotion_scores(sentence))

[{'label': 'sadness', 'score': 0.8478295803070068}, {'label': 'joy', 'score': 0.0340697206556797}, {'label': 'love', 'score': 0.0015943567268550396}, {'label': 'anger', 'score': 0.09716669470071793}, {'label': 'fear', 'score': 0.01846015267074108}, {'label': 'surprise', 'score': 0.000879530212841928}]
